Analyze various classical quantities in a single simulation. You need to store "data". It assumes a fixed rmax.

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots; pythonplot()
using DelimitedFiles
using Interpolations
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
base = "../examples/classical_runs/"
your_dir = base*"a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_true_0.001_overMp2_25.132741228718345_damp0"

# convention of 1/M_Planck^2; set manually
oMp2 = 25.132741228718345

In [ ]:
# resolution
D = 5
Nr = 128*2^D + 3 

# load the x grid
r = h5read(your_dir*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its, all_filenames) = list_h5_files(your_dir*"/data_$(Nr)", prefix="data_")
println(length(its))

In [ ]:
#it is the timestep we look at
i = 65
it = its[i]
it_str  = lpad(it, 4, "0")

v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
v = h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")
t = h5readattr(your_dir*"/data_$(Nr)/data_$(it_str).h5", "/")["time"]

Φ    = v[:,1]
Π    = v[:,2]
Ψ    = v[:,3]
A    = v[:,4]
B    = v[:,5]
DB   = v[:,6]
Utld = v[:,7]
K    = v[:,8]
KB   = v[:,9]
λ    = v[:,10]
α    = v[:,11]
Dα   = v[:,12]
Θ    = v[:,13]
Zr   = v[:,14];


The lapse at r=0, the location of the apparent horizon, and its area:

In [ ]:
length(its)
αt_r0 = zeros(length(its))
AH_loc = zeros(length(its))
AH_area = zeros(length(its))

tt = zeros(length(its))
for i in 1:length(its)
    it = its[i]
    it_str  = lpad(it, 4, "0")
    αt_r0[i] = h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[1,11]
    tt[i] = h5readattr(your_dir*"/data_$(Nr)/data_$(it_str).h5", "/")["time"][1]

    A     =  h5read(your_dir*"/data_$((Nr-3)+ 3)/data_$(it_str).h5","v")[:,4]
    B     =  h5read(your_dir*"/data_$((Nr-3)+ 3)/data_$(it_str).h5","v")[:,5]    
    KB     =  h5read(your_dir*"/data_$((Nr-3)+ 3)/data_$(it_str).h5","v")[:,9]    
    AH = find_AH(r, B, A, KB)

    AH_loc[i] = AH

    # the interpolation is needed because we take AH to be
    # inbetween the two grid points where the sign of the expansion of null rays changes
    B_intp = cubic_spline_interpolation(range(-2*dr, r[end], length=length(r)), B[:])
    AH_area[i] = 4*π*AH^2*B_intp(AH)
end


In [ ]:
plot(tt[1:end], αt_r0[1:end], label="α(t, r=0)", linewidth=3,
    frame=true, xaxis="time", title="r=0", xlim=(tt[1],tt[end]))

In [ ]:
plot(tt[1:end], AH_loc[1:end], label=L"r_{AH}(t)", linewidth=3, legend=:topleft,
    frame=true, xaxis="time", title="Apparent horizon location", 
    yaxis=L"r_{AH}", xlim=(tt[1],tt[end]))

In [ ]:
plot(tt[1:end], AH_area[1:end], label=L"A_{AH}(t)", linewidth=3, legend=:bottomright,
    frame=true, xaxis="time", title="Apparent horizon area", 
    yaxis=L"A_{AH}", xlim=(tt[1],tt[end]))

In [ ]:
# get index of r_c in coarse resol, for which  r[index] is the first causally connected point
function find_ri_max_causal(your_dir::String, Nr::Int, its::Array)
    it      = its[end]
    it_str  = lpad(it, 4, "0")
    tc_end  =  h5readattr(your_dir*"/data_$(Nr)/data_$(it_str).h5", "/")["time"]
    ii = 1
    r_c = h5read(your_dir*"/data_$(Nr)/r.h5","r")
    while r_c[ii] != r_c[end] - tc_end
        ii = ii + 1
    end
    ri_max_causal = ii-1
    drc = r_c[4] - r_c[3]
    r_c[ri_max_causal] == r_c[end] - tc_end - drc
    return ri_max_causal
end

ri_max_causal = find_ri_max_causal(your_dir, Nr, its)

In [ ]:
ri_min = 3
ri_max = ri_max_causal

AH = find_AH(r, B, A, KB)
println("Apparent horizon is at r = ", AH)

H, P = SpheriCo.classical.constraints(v, dr, oMp2);
# calculate the Ricci scalar
R  = SpheriCo.Ricci_scalar(v, r, oMp2);

plot(r[ri_min:ri_max], Φ[ri_min:ri_max], title = "t=$(t)",  label=L"\Phi(r)", linewidth = 3, frame = true,
    xlim=(r[ri_min], r[ri_max]), xaxis=nothing)
p1 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black")
    
plot(r[ri_min:ri_max], abs.(R[ri_min:ri_max]), label=L"|R(r)|", linewidth=3, frame=true,
    xlim=(r[ri_min], r[ri_max]), xaxis=nothing)
p2 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black") 

plot(r[ri_min:ri_max], α[ri_min:ri_max], label=L"\alpha(r)", linewidth=3, frame=true,
    xlim=(r[ri_min], r[ri_max]), xaxis=nothing)
p3 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black") 

plot(r[ri_min:ri_max], abs.(H[ri_min:ri_max]), label=L"|H(r)|", linewidth=3, frame=true,
    xlim=(r[ri_min], r[ri_max]), xaxis="r")
p4 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black")

plot(r[ri_min:ri_max], abs.(P[ri_min:ri_max]), label=L"|P(r)|", linewidth=3, frame=true,
    xlim=(r[ri_min], r[ri_max]), xaxis="r")
p5 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black") 
  
plt = plot(p1, p2, p3, p4, p5, layout = grid(3, 2), wsize = (800,600))
#savefig(plt, "./classical_experiments/"*par*".pdf")

In [ ]:
function Φ_α_H_P_ric_evol(i::Int, ri_min, ri_max)
    it = its[i]
    it_str  = lpad(it, 4, "0")
    
    v = h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")
    t = h5readattr(your_dir*"/data_$(Nr)/data_$(it_str).h5", "/")["time"]
    
    Φ    = v[:,1]
    A    = v[:,4]
    B    = v[:,5]
    KB   = v[:,9]
    α    = v[:,11]
    
    AH = find_AH(r, B, A, KB)
    
    H, P = SpheriCo.classical.constraints(v, dr, oMp2);
    # calculate the Ricci scalar
    R  = SpheriCo.Ricci_scalar(v, r, oMp2);
    
    plot(r[ri_min:ri_max], Φ[ri_min:ri_max], title = "t=$(t)",  label=L"\Phi(r)", linewidth = 3, frame = true,
        xlim=(r[ri_min], r[ri_max]), xaxis=nothing)
    p1 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black")
        
    plot(r[ri_min:ri_max], abs.(R[ri_min:ri_max]), label=L"|R(r)|", linewidth=3, frame=true,
        xlim=(r[ri_min], r[ri_max]), xaxis=nothing)
    p2 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black") 
    
    plot(r[ri_min:ri_max], α[ri_min:ri_max], label=L"\alpha(r)", linewidth=3, frame=true,
        xlim=(r[ri_min], r[ri_max]), xaxis=nothing)
    p3 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black") 
    
    plot(r[ri_min:ri_max], abs.(H[ri_min:ri_max]), label=L"|H(r)|", linewidth=3, frame=true,
        xlim=(r[ri_min], r[ri_max]), xaxis="r")
    p4 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black")
    
    plot(r[ri_min:ri_max], abs.(P[ri_min:ri_max]), label=L"|P(r)|", linewidth=3, frame=true,
        xlim=(r[ri_min], r[ri_max]), xaxis="r")
    p5 = plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black") 
      
    plt = plot(p1, p2, p3, p4, p5, layout = grid(3, 2), wsize = (800,600))
end

In [ ]:
anim = @animate for i ∈ 1:length(its)
    Φ_α_H_P_ric_evol(i, 3, ri_max_causal)
end
gif(anim, "Φ_α_H_P_ric_evol.gif", fps = 5)

In [ ]:
function f_evol(i::Int, fi::Int, ri_min, ri_max)
    it = its[i]
    it_str  = lpad(it, 4, "0")
    
    A   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,4]
    B   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,5]
    KB   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,9]

    f   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,fi]
    t   =  h5readattr(your_dir*"/data_$(Nr)/data_$(it_str).h5", "/")["time"]
    
    plot(r[ri_min:ri_max], f[ri_min:ri_max], title = "t=$(t)", label=v_classic_labels[fi], linewidth=3, frame=true, xaxis="r")
    plot!([AH], seriestype="vline", label="AH", linewidth=1, color = "black") 
    
end



In [ ]:
anim = @animate for i ∈ 1:length(its)
    f_evol(i, 1, 3, ri_max_causal)
end
gif(anim, "f_evol.gif", fps = 5)